# NER Spacy model,  with ready data, for few categories

Plan:
    - load the fully ready sentences
    - count how many are for each category (cell, protein, gene)
    - make training / test set with entities labelled for spacy 
    - 
    - ...
    - train bert on the same set and compare 
    

 https://medium.com/@nikkisharma536/how-to-create-custom-ner-in-spacy-cfcd531f8773

1. [Define entities of interest.](#fyrsta)
2. [Load the newest tagged set.](#annað)
3. [Select fully labelled sentences.](#þriðja)
4. [Transform into Spacy NER format.](#fjórða)
5. [Split into training / test sets.](#fimmta)
6. [Load scispacy model. Train (for a few shuffled iterations).](#sjötta)
7. [Test!](#sjöunda)
8. [Try scispacy's NER models](#áttunda)

In [61]:
# Load wikilabelled and use it to tag the newet set.

import json 

with open(os.path.dirname(os.getcwd()) + '/wiki_labelled.json') as f:
    wikilab = json.load(f)
    print(len(wikilab))

341


1. Define classes and update the newest tagged set <a name="fyrsta"></a>

In [1]:
# define entities of interest

interesting = ['cell_line', 'science_word', 'reagent', 'gene']
#interesting = ['number', 'unit', 'table_cell']

In [62]:
# mark unambiguous terms from  from wikilabelled

catz = []
n_wikicats = []
ambiguous = []
#new_file = open('SPACY NER/wiki_unambiguous', 'w') 


with open('SPACY NER/imbalanced3') as cells:

    # label everything else with an 'O'
    
    for l in cells.readlines():
        items = l.split(',')
        if len(items) == 5:
            # skip strange rows 
            word = items[2]
            cat = items[4].strip()
            catz.append(cat)
                
            if word in [x[0] for x in wikilab]:
                # been wikilabelled
                if cat == 'X':
                    # was unlabelled
                    its_cat = [x[1] for x in wikilab if x[0] == word]
                    if len(its_cat) == 1:
                        # unambiguously labelled 
                        items[4] = its_cat[0]
                        
                        l = ','.join(items) + '\n'
                    else:
                        ambiguous.append((word,its_cat))
#            new_file.write(l)

            

In [9]:
unlabelled = count_unknowns('SPACY NER/wiki_ambiguous')
len(unlabelled)

826

In [4]:
import os
os.getcwd()

'/Users/valdimareggertsson/Documents/Valdi/Vetrarönn 2019/NER/Spacy NER'

In [3]:

def count_unknowns(path_to_csv):

    unknowns = []

    with open(path_to_csv) as labelled_data:


        for l in labelled_data.readlines():
            items = l.split(',')

            if len(items) == 5:
                # skip strange rows 

                new_sentence_nr = items[1]
                word = items[2]
                category = items[4].strip()

                if category == 'X':
                    unknowns.append(word)


    unknowns = sorted(list(set(unknowns)))

    return unknowns


len(count_unknowns('SPACY NER/wiki_unambiguous'))

857

### 2. Take the marked sentences from the categories of interest <a name="annað"></a>

In [4]:
import os

unlabelled = count_unknowns('SPACY NER/imbalanced')

In [5]:
### Get unlabelled and labelled

sentences_with_unlabelled = []
sentence_ids = [] 
with open('SPACY NER/imbalanced2') as cells:
    
    # skip the header 
    for l in cells.readlines()[1:]:
        items = l.split(',')
        
        for i in unlabelled:
            if i in l:
                sentences_with_unlabelled.append(items[1])
#                print(items[1])

        if len(items) == 5:
            sentence_ids.append(items[1])
                

In [6]:
ready_ids = sorted(list(set([int(x) for x in sentence_ids if x not in sentences_with_unlabelled])))
ready_ids

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [7]:
len(ready_ids)

17705

In [9]:
ready_words = []
with open('SPACY NER/imbalanced2') as f:
    
    for l in f.readlines()[1:]:
        items = l.split(',')
        
        if len(items) == 5:
            if int(items[1]) in ready_ids:
                ready_words.append(items)
                

### 3. Filter for fully labelled sentences <a name="þriðja"></a>

In [10]:
# filter for fully labelled sentences containing the entities 

current_sentence = '0'
ready_s = [[]]
ready_labels = [[]] 

for w in ready_words:
    if w[1] == current_sentence:
        ready_s[-1].append(w[2])
        ready_labels[-1].append(w[4].strip())

    else:
        current_sentence = w[1]
        ready_s.append([])
        ready_labels.append([])
        
        

Now we have ready sentences and their corresponding labels.

In [11]:
ready_s

[['A1', '66056', '.', 'A2', '5.11'],
 ['2',
  '.',
  'Cm',
  'R',
  '=',
  '2.5',
  '.',
  'Nice',
  'calcium',
  'current',
  'and',
  'transient',
  '.',
  'Caffeine',
  'was',
  'beautiful',
  '.',
  'Isoprenaline',
  'was',
  'also',
  'done',
  'but',
  'it',
  "'s",
  'important',
  'to',
  'note',
  'that',
  'the',
  'isoprenaline',
  'stock',
  'was',
  'made',
  'from',
  'the',
  'old',
  'stock',
  'which',
  'might',
  'reflect',
  'in',
  'the',
  'reduced',
  'activation',
  'of',
  'beta',
  'adrenergic',
  'receptors',
  '.',
  'Leak',
  'was',
  'also',
  'attempted',
  'nice',
  'effective',
  'tetracaine',
  'was',
  'seen',
  'but',
  'it',
  'seems',
  'the',
  'is',
  'not',
  'properly',
  'working',
  'as',
  'expected',
  'probably',
  'due',
  'to',
  'a',
  'low',
  'perfusion',
  'pressure',
  '.',
  'However',
  'a',
  'nice',
  'leak',
  'experiment',
  'was',
  'done',
  '.'],
 ['temperature',
  'for',
  'amplification',
  'was',
  'achieved',
  'in',
  

In [12]:
len(ready_s)

n_genes,n_cell_lines, n_proteins, n_reagents,n_science = 0,0,0,0,0
for i,s in enumerate(ready_s):
    for j,w in enumerate(s):
        if ready_labels[i][j] == 'gene':
            n_genes += 1 
            break
        
        if ready_labels[i][j] == 'cell_line':
            n_cell_lines += 1 
            break
        if ready_labels[i][j] == 'science_word':
            n_science += 1 
            break
        
        if ready_labels[i][j] == 'protein':
            n_proteins += 1 
            break
            
        if ready_labels[i][j] == 'reagent':
            n_reagents += 1 
            break
        

print(n_genes,n_cell_lines, n_proteins, n_reagents, n_science)

4030 3999 0 2169 3069


Ok there are a decent amount of sentences for the interesting categories...

### 4. Make spacy training data <a name="fjórða"></a>

In [13]:
# transform sentences with entities of interest into the spacy format

sentences = [[]]
entities = []
tagger = {"entities": []}
old_sentence_nr = '0'

# skip the header 
for i,sent in enumerate(ready_s):

    for j,w in enumerate(sent):
      #  print(w)
        category = ready_labels[i][j]
        
 

        start_index = sum([len(x) +1 for x in sent[:j]])
        if category in interesting:
            tagger["entities"].append((start_index, start_index + len(w), category))
   # print('')
    # loop over the sentence and mark the entities:
    #entities.append(tagger)

    entities.append((' '.join(sent),tagger))
    tagger = {"entities": []}



            
print(len(entities))
        
# sentences are the sentences pieced together again as lists 


17705


In [14]:
for e in entities:
    print(e,'\n')

('A1 66056 . A2 5.11', {'entities': []}) 

("2 . Cm R = 2.5 . Nice calcium current and transient . Caffeine was beautiful . Isoprenaline was also done but it 's important to note that the isoprenaline stock was made from the old stock which might reflect in the reduced activation of beta adrenergic receptors . Leak was also attempted nice effective tetracaine was seen but it seems the is not properly working as expected probably due to a low perfusion pressure . However a nice leak experiment was done .", {'entities': []}) 

('temperature for amplification was achieved in D2  E2  F2  G2  and H2 .', {'entities': []}) 

('µl 2 N NaOH into dilution .', {'entities': [(7, 11, 'reagent')]}) 

('is added to the plate and put on a plate shaker at 450 rpm .', {'entities': []}) 

('gel with first lane 10 µl precision plus protein unstained standard from BioRad catalog number 161 - 036310 .', {'entities': []}) 

('µl to lane 1 .', {'entities': []}) 

('A1 to A9 is my serial dilution of standard w

('PLT legend with English radio in the background . Extraction . Extraction . The biphasic solution was stirred at room temp for 20 min and then transferred into a separation funnel . The phase cut is clear and fast . The clear yellow solution was dried on the water bath up at 50 ℃ and 10 mbar to give . Test end .', {'entities': [(0, 3, 'gene')]}) 

('19063 - 4 has a lot of red dots that looks like background .', {'entities': []}) 

('mixed up a new agarose gel with 1 % agarose in TAE buffer', {'entities': []}) 

('PCR with new primers and RVG mScarlet plus  Amnion . Number 52 for mScarlet and 61 for mScarlet  and number 42 for gm mScarlet Amnion .', {'entities': []}) 

('use 5 ml instead of 10 ml for the sample 13 .', {'entities': []}) 

('T is being prepped for bioanalyzer by being resuspended to a concentration of 200 ng/µl', {'entities': []}) 

('HM 5264 all 4 mice look good  healthy running around .', {'entities': []}) 

('to steps 3 .', {'entities': []}) 

('HeLa cells at the 96 

("make the SB stock . There 's a 10 mg powder add 2.6 ml of DMSO for the new 50 mg powder need 13 ml DMSO . Make 500 µl aliquots .", {'entities': [(58, 62, 'reagent'), (99, 103, 'reagent')]}) 

("tomorrow do n't forget to run the gel . To check the AAV9 plasmids from .", {'entities': [(58, 66, 'science_word')]}) 

('supernatant resuspending cells with 500 µl MACS buffer . Going to sorting sorting on CD 102 High ; CD 140a low population .', {'entities': [(43, 47, 'gene')]}) 

('30 µl of each in HPLC vial . PL-19 - 1088 : 7.5 ml final volume . PB 19 1089 : almost 10 ml . PB 19 1090 7.5 ml .', {'entities': []}) 

('me to select my infected BJAB cells . With hygromycin 200 µg / ml . On Friday evening at 4:00 PM .', {'entities': [(25, 29, 'cell_line')]}) 

('buffer . 20 mM HEPES  150 mM sodium chloride  1 Milimolar TCEP .', {'entities': []}) 

('PBS used as for injection number . Falciparum infection from Friday 2nd of November .', {'entities': [(0, 3, 'reagent')]}) 

('pipetted the DNA int


('96 h time point was washed 1 x with PBS . The PBS was removed and the plate was placed at -80 ° .', {'entities': [(36, 39, 'reagent'), (46, 49, 'reagent')]}) 

('cells for the IT 25834 HepG2 C3a spheroid assay following a single vial and .', {'entities': [(23, 28, 'cell_line'), (29, 32, 'cell_line'), (33, 41, 'science_word')]}) 

('1 spr assay using', {'entities': [(2, 5, 'cell_line')]}) 

("make my virus dilutions  I will take 280 μl of virus stock . And add this to . μl . Of PBS . That gives me - 2 dilution . I will then take 280 μl of that and add it to μl of PBS to give them -4 and then the same together -6 dilution full stop . That will give me my -2 -4 -6 donation and I live there and remove . ? cell and add 200 μl to each well . In the corresponding -2 -4 -6 wells . So should be adding virus to all of them and then I will put the ? cell back on top . I want to make sure that I add extra pen - strep and gentamicin . To the ? cell before add it back on to make sure it 's killin

('cell viability is 65.3 % and concentration is 4.4 * 10 ^ 5 cells per ml .', {'entities': []}) 

("I 'm preparing samples for our needs it . And first I need to prepare the dilutions for the RNA . So the RNA does n't stand too long on ice to prevent degradation . And um . Then I need to . So first prepare the water  then prepare the master mixes so with oligo DTs  Dntps and RNase Inhibitors and the second mastermix that contains RT reaction . Without the reverse transcriptase that will come the last . Once I have prepared that  then I pick up 30 samples make dilutions . And start typing basically to do the binding with oligos . Factors that Add RT reaction and run the pcr .", {'entities': [(257, 262, 'science_word'), (528, 534, 'science_word'), (578, 581, 'science_word')]}) 

('homogenised sample number 2 is very yellow compared to the others that are dark red .', {'entities': [(0, 11, 'science_word')]}) 

("some reason  it looks like the HL ADR is showing up very oddly in the CD ADP 


('pipette my 71.43 µl of my phospho - RAD antibody number 4223 I use the P200 pipette Blue C  serial number U545796 calculation number DOB 9 URYCXE .', {'entities': [(71, 75, 'cell_line')]}) 

('Sf9 splitting .', {'entities': [(0, 3, 'cell_line')]}) 

(' we starting to pipette the experiment at 1:30 . We hope to be done with premixing all the reagents by 3:00 in the afternoon and pipet on the cells  so that we have at least 22 h incubation over night .', {'entities': [(73, 82, 'science_word')]}) 

('pipetted 5 µl into a Falcon tube . I then put the tubes into the water bath . The results show that the water bath was not hot enough .', {'entities': [(0, 8, 'science_word')]}) 

('1 . Cm = 99 . Series resistance was a little high . The current and transient transient were good even though there was leak in the L - type Calcium current . Caffeine was also applied with nice NCX and caffeine induced transient .', {'entities': [(195, 198, 'gene')]}) 

('protein is at 5 mg / mill .', {'entiti


('next plans are to split the cells SF9 ESF as well as SF9 Lonza to one million cells per milliliter and later on also produce P1 virus .', {'entities': [(34, 37, 'cell_line'), (53, 56, 'cell_line')]}) 

('study was designed to maximize the likelihood to show pharmacological effect and the target was to have compound exposure above 3 X EC50 drawing the complete dosing interval', {'entities': [(132, 136, 'cell_line')]}) 

('will treat the day cells today with the life dead indicator and the caspase 3 assay around 11 a.m. We need to ensure a one hour incubation with the caspase according to the manual and we would like to measure the cells around 1 p.m. So  if we aim for 11 p.m. we have a safety margin for pipetting errors as well as we takes longer . I will discuss with Caroline if she pipet this or if I pipet this . [ inaudible ] options are fine . Carolyn will for sure take care of the Qbeads part where they going to measure all the cytokines in parallel of the cells .', {'entities':

('supernatant from tubes A1 to 12 and put into B1 to 12 . Added 500 µl of PBS to A5 to 12 . Vortex . Removed 10 µl .', {'entities': [(72, 75, 'reagent')]}) 

('M1 cells cultured in 4 ml media .', {'entities': []}) 

("at 9:15 sonication was finished so the whole procedure for 6 tubes takes about 1 h  but it 's also a lot of incubation time with just resuspending the pellets .", {'entities': [(8, 18, 'science_word')]}) 

("cells were at 0.68 and 86 % . I 'm going to split them so that 7 . of 10 mL of Jurkat cells goes into 25 mL of new cells . Actually that might be wrong . Let me check .", {'entities': [(79, 85, 'cell_line')]}) 

('696  no block neuro D1  1 to 100 in TBST  looking at human pancreas .', {'entities': [(14, 19, 'gene')]}) 

('number of viable cells of HepG2 is . I would add of cell suspension and 70 mL of media to get of total cell suspension to seed into 5 plates .', {'entities': [(26, 31, 'cell_line')]}) 

('BJAB cells infected with the RRV AELAAN mutant  the wild type 

('HSD 1012 .', {'entities': [(0, 3, 'gene')]}) 

('test PLT legend with loud English talking in the background . Extraction . The biphasic solution was stirred at room temp for 20 min and then transferred into a separation funnel . Phase cut is clear and fast . The clear yellow solution was dried on the water bath of up at 50 ℃ and 10 mbar to give . Test end dragonfly .', {'entities': [(5, 8, 'gene')]}) 

('substrate has been added to the wells and the plate is put on a orbital Shaker at 450 RPM .', {'entities': []}) 

('pipetted 5 µl into 5ml Falcon tube and then I transfer the media into 69 well plate .', {'entities': [(0, 8, 'science_word')]}) 

('BLIMP 1 spr assay . Buffer is 20 mM HEPES  150 mM sodium chloride  1 mM TCEP  0.05 % Tween-20  1 % DMSO pH 7.5', {'entities': [(8, 11, 'cell_line'), (85, 93, 'reagent'), (99, 103, 'reagent')]}) 

('the purification after restriction of L1RFP amp ori with the general selection cassette .', {'entities': [(48, 51, 'gene')]}) 

('BJAB infectio


('number 10 . Concentration 259.9 ng/µl . Fraction A 1.87 . Fraction B 2.38 .', {'entities': []}) 

('count for HepG2 C3A cells is 2.09 million total  1.85 million viable cells  88.6 % viability .', {'entities': [(10, 15, 'cell_line'), (16, 19, 'cell_line')]}) 

('round . PBS 32 mosquitoes .', {'entities': [(8, 11, 'reagent')]}) 

("for luminescence reads is in MBP / Cell SAR . And it 's called IT 17002 Mitotox 384 CTG", {'entities': [(29, 32, 'gene')]}) 

('pipetted 5 µl into a Falcon tube .', {'entities': [(0, 8, 'science_word')]}) 

("Multidrop washed with PBS and now going to dispense cell suspension for the phospholipidosis assay using the 384 Standard cassette . Volume is 30 µl . I 'm dispensing into the full plate . Priming with cell suspension .", {'entities': [(22, 25, 'reagent')]}) 

(' prepare 2 times PBS buffer with 1 molar sodium chloride using 2 packages of Sigma PBS P3813 .', {'entities': [(17, 20, 'reagent'), (83, 86, 'reagent')]}) 

('HepG2 C3A cells at 300 rcf for 3 

### 5. Make training/test set <a name="fimmta"></a>

In [15]:
# Make training / test

import random
random.shuffle(entities)

# þjálfa á 3200, testa á 622 

train_set = entities[:15000]
test_set = entities[15000:]

### 6. Load spacy model and train on our data. <a name="sjötta"></a> 

In [16]:
c

Created blank 'en' model
Starting iteration0


KeyboardInterrupt: 

### 7. Test. <a name="sjöunda"></a>

In [23]:
# Testa, eins og í tutorialinu?
# Fá confusion matrix?
# nota scorer.scores

import spacy
from spacy.gold import GoldParse
from spacy.scorer import Scorer

def evaluate(model, examples):
    scorer = Scorer()
    for input_, annot in examples:
        if len(input_) > 3:
            doc_gold_text = model.make_doc(input_)
            gold = GoldParse(doc_gold_text, entities=annot['entities'])
            pred_value = model(input_)
            scorer.score(pred_value, gold)
    return scorer.scores

#test_result = evaluate(nlp, test_set)

In [31]:
test_result

{'uas': 0.0,
 'las': 0.0,
 'las_per_type': {'root': {'p': 0.0, 'r': 0.0, 'f': 0.0}},
 'ents_p': 64.70588235294117,
 'ents_r': 91.66666666666666,
 'ents_f': 75.86206896551724,
 'ents_per_type': {'gene': {'p': 100.0, 'r': 60.0, 'f': 74.99999999999999},
  'reagent': {'p': 84.61538461538461, 'r': 100.0, 'f': 91.66666666666666},
  'cell_line': {'p': 66.66666666666666, 'r': 100.0, 'f': 80.0},
  'ENTITY': {'p': 0.0, 'r': 0.0, 'f': 0.0}},
 'tags_acc': 0.0,
 'token_acc': 100.0,
 'textcat_score': 0.0,
 'textcats_per_cat': {}}

#### Compare with their NER model

In [32]:
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.4/en_ner_craft_md-0.2.4.tar.gz

     |████████████████████████████████| 70.1MB 706kB/s eta 0:00:01
  Stored in directory: /Users/valdimareggertsson/Library/Caches/pip/wheels/26/69/af/fab46820818e775d68792af96d74241122251926f713346657
Successfully built en-ner-craft-md


In [82]:
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.4/en_ner_jnlpba_md-0.2.4.tar.gz

!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.4/en_ner_bionlp13cg_md-0.2.4.tar.gz

     |████████████████████████████████| 70.1MB 23.1MB/s eta 0:00:01     |██████████████████████████████▊ | 67.2MB 23.1MB/s eta 0:00:01
  Stored in directory: /Users/valdimareggertsson/Library/Caches/pip/wheels/f5/04/a5/5184d678ecc10d561facfa550b6db7dd0cfc05f425f643ed01
Successfully built en-ner-jnlpba-md
     |████████████████████████████████| 70.1MB 1.1MB/s eta 0:00:01     |███████████                     | 23.9MB 9.9MB/s eta 0:00:05


  Stored in directory: /Users/valdimareggertsson/Library/Caches/pip/wheels/ef/2a/a6/014a4ac046ce7e0afeba214dc8ca1ab1bd5cbf05c4b77b28ec
Successfully built en-ner-bionlp13cg-md


### 8. Try scispacy's special bio ner models <a name="áttunda"></a>

In [18]:
import spacy
#nerd = train_model('en_ner_craft_md',train_set,5)
#test_result2 = evaluate(nerd, test_set)

In [24]:
#jnlpba = train_model('en_ner_jnlpba_md',train_set,5)
test_result3 = evaluate(jnlpba, test_set)

In [25]:
test_result3

{'uas': 0.0,
 'las': 0.0,
 'las_per_type': {'root': {'p': 0.0, 'r': 0.0, 'f': 0.0},
  'compound': {'p': 0.0, 'r': 0.0, 'f': 0.0},
  'agent': {'p': 0.0, 'r': 0.0, 'f': 0.0}},
 'ents_p': 67.3913043478261,
 'ents_r': 83.78378378378379,
 'ents_f': 74.6987951807229,
 'ents_per_type': {'gene': {'p': 57.14285714285714,
   'r': 80.0,
   'f': 66.66666666666666},
  'cell_line': {'p': 64.28571428571429,
   'r': 81.81818181818183,
   'f': 72.00000000000001},
  'science_word': {'p': 57.14285714285714,
   'r': 66.66666666666666,
   'f': 61.53846153846153},
  'reagent': {'p': 90.9090909090909, 'r': 100.0, 'f': 95.23809523809523}},
 'tags_acc': 0.0,
 'token_acc': 100.0,
 'textcat_score': 0.0,
 'textcats_per_cat': {}}

In [28]:
#bionlp = train_model('en_ner_bionlp13cg_md',train_set,5)
test_result4 = evaluate(bionlp, test_set)

In [29]:
test_result4

{'uas': 0.0,
 'las': 0.0,
 'las_per_type': {'root': {'p': 0.0, 'r': 0.0, 'f': 0.0},
  'compound': {'p': 0.0, 'r': 0.0, 'f': 0.0},
  'agent': {'p': 0.0, 'r': 0.0, 'f': 0.0}},
 'ents_p': 82.35294117647058,
 'ents_r': 75.67567567567568,
 'ents_f': 78.87323943661971,
 'ents_per_type': {'gene': {'p': 77.77777777777779,
   'r': 70.0,
   'f': 73.68421052631578},
  'cell_line': {'p': 75.0, 'r': 81.81818181818183, 'f': 78.26086956521738},
  'reagent': {'p': 100.0, 'r': 100.0, 'f': 100.0},
  'science_word': {'p': 66.66666666666666,
   'r': 33.33333333333333,
   'f': 44.44444444444444}},
 'tags_acc': 0.0,
 'token_acc': 100.0,
 'textcat_score': 0.0,
 'textcats_per_cat': {}}

In [4]:
unlabelled = count_unknowns('SPACY NER/wiki_ambiguous')

In [5]:
len(unlabelled)

826

In [7]:
len(count_unknowns('SPACY NER/wiki_ambiguous2'))

814

In [9]:
len(count_unknowns('SPACY NER/imbalanced'))

0

### Try labelling the imbalanced data with en_core_sci_lg

In [12]:
# load data

import pandas as pd

bucket='biobert-labtwin'
data_key = 'imbalanced'
#data_location = 's3://{}/{}'.format('biobert-labtwin', data_key)
data_location = 'SPACY NER/imbalanced'

df_data = pd.read_csv(data_location).fillna('')

df_data.head(n=20)

# Have a look TAG cat
df_data.category.unique()

# Analyse the Tag distribution
df_data.category.value_counts()

O               243708
cell_line         3131
science_word      2683
gene              2636
reagent           1682
Name: category, dtype: int64

In [13]:
ready_words = []
with open('SPACY NER/imbalanced') as f:
    
    for l in f.readlines()[1:]:
        items = l.split(',')
        
        if int(items[1]) in ready_ids:
            ready_words.append(items)
                

NameError: name 'ready_ids' is not defined